In [20]:
import os
from pyspark.sql import SparkSession, functions as F

# paths (update if needed)
INPUT_PATH = r"C:\Users\basun\OneDrive\Desktop\Programming practice\bigdata_project\data\nyc_taxi\yellow_tripdata_2019-01.parquet"
OUTPUT_DIR = r"C:\Users\basun\OneDrive\Desktop\Programming practice\bigdata_project\outputs"
CURATED_PATH = os.path.join(OUTPUT_DIR, "curated_parquet")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print("Inputs set ✅")

Inputs set ✅


In [21]:
spark = SparkSession.builder.appName("BigDataAnalysis").getOrCreate()
print("SparkSession ready ✨", spark.version)

SparkSession ready ✨ 4.0.0


In [22]:
df = spark.read.parquet(INPUT_PATH)

# check schema + preview
df.printSchema()
df.show(5)
print("Raw row count:", df.count())

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+---

In [23]:
from pyspark.sql.functions import to_timestamp, col, unix_timestamp, hour, dayofmonth, dayofweek

df = (df
      .withColumn("tpep_pickup_datetime", to_timestamp(col("tpep_pickup_datetime")))
      .withColumn("tpep_dropoff_datetime", to_timestamp(col("tpep_dropoff_datetime")))
     )

df = df.withColumn(
    "duration_min",
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60.0
)

# filters (relaxed to avoid empty df issues)
df = df.filter(
    (col("trip_distance") > 0) &
    (col("trip_distance") < 100) &
    (col("duration_min") > 0) &
    (col("duration_min") < 500)
)

df = (df
      .withColumn("hour", hour(col("tpep_pickup_datetime")))
      .withColumn("day", dayofmonth(col("tpep_pickup_datetime")))
      .withColumn("dow", dayofweek(col("tpep_pickup_datetime")))
     )

df.show(5)
print("Filtered row count:", df.count())


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+----+---+---+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|      duration_min|hour|day|dow|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+----+---+---+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1

In [24]:
# install pandas inside notebook (only once per env)
!pip install pandas

import pandas as pd
import os
from pyspark.sql import functions as F

# Percentiles
q_td = df.approxQuantile("trip_distance",[0.5,0.95],0.01)
q_du = df.approxQuantile("duration_min",[0.5,0.95],0.01)

hourly = df.groupBy("hour").count().orderBy("hour")
top_pu = df.groupBy("PULocationID").count().orderBy(F.desc("count")).limit(10)

q_pdf = pd.DataFrame([
    {"column":"trip_distance","p50":q_td[0],"p95":q_td[1]},
    {"column":"duration_min","p50":q_du[0],"p95":q_du[1]}
])
hourly_pdf = hourly.toPandas()
top_pu_pdf = top_pu.toPandas()

print("Percentiles:\n", q_pdf)
print("\nHourly volume:\n", hourly_pdf)
print("\nTop pickup zones:\n", top_pu_pdf)

# save
q_pdf.to_csv(os.path.join(OUTPUT_DIR, "percentiles.csv"), index=False)
hourly_pdf.to_csv(os.path.join(OUTPUT_DIR, "hourly_volume.csv"), index=False)
top_pu_pdf.to_csv(os.path.join(OUTPUT_DIR, "top_pickups.csv"), index=False)
print("✅ Saved CSVs to:", OUTPUT_DIR)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Percentiles:
           column        p50        p95
0  trip_distance   1.520000  11.100000
1   duration_min  10.233333  31.316667

Hourly volume:
     hour   count
0      0  205281
1      1  147303
2      2  107786
3      3   76657
4      4   60033
5      5   74058
6      6  176053
7      7  301419
8      8  370458
9      9  362755
10    10  358095
11    11  371960
12    12  397400
13    13  400173
14    14  428754
15    15  447962
16    16  416246
17    17  464137
18    18  511222
19    19  471618
20    20  419865
21    21  406520
22    22  365937
23    23  279183

Top pickup zones:
    PULocationID   count
0           237  331206
1           236  321714
2           161  310856
3           162  275715
4           230  261970
5           186  259110
6            48  239520
7           170  238063
8           234  236469
9           142  234237
✅ Saved CSVs to: C:\Users\basun\OneDrive\Desktop\Programming practice\bigdata_project\outputs


In [25]:
df2 = df.repartition(8).persist()
import time
t0 = time.time()
_ = df2.agg(F.avg("fare_amount").alias("avg_fare")).collect()
print(f"Spark avg(fare_amount) with 8 partitions took {time.time()-t0:.2f}s")


Spark avg(fare_amount) with 8 partitions took 0.77s


In [26]:
print(OUTPUT_DIR)

C:\Users\basun\OneDrive\Desktop\Programming practice\bigdata_project\outputs
